<h3> P1shapes </h3>

In [1]:
def P1shapes(r, s):
    """
        Funkcja obliczająca w punkcie (r,s) wartości 3 funkcji kształtu P1 oraz ich pochodne cząstkowe.
        input: r,s - float(), float(), współrzędne punktu
        output: S - np.array(), wektor wartości 3 funkcji kształtu w (r,s);
                dSdr - np.array(),  wektor pochodnej czastkowej po 1. współrzędnej;
                dSds - np.array(), wektor pochodnej czastkowej po 2. współrzędnej
    """

    S = np.array([1 - r - s, r, s])
    dSdr = np.array([-1, 1, 0])
    dSds = np.array([-1, 0, 1])

    return S, dSdr, dSds


<h3> P2shapes </h3>

In [2]:
def P2shapes(r,s):
    """
        Funkcja obliczająca w punkcie (r,s) wartości 3 funkcji kształtu P2 oraz ich pochodne cząstkowe.
        input: r,s - float(), float(), współrzędne punktu
        output: S - np.array(), wektor wartości 3 funkcji kształtu w (r,s);
                dSdr - np.array(),  wektor pochodnej czastkowej po 1. współrzędnej;
                dSds - np.array(), wektor pochodnej czastkowej po 2. współrzędnej
    """
    
    S = np.array([ 1 - 3*r - 3*s + 2*r**2 + 4*r*s + 2*s**2, 2*r**2 - r,2*s**2 - s,
                  4*r*s,4*s - 4*r*s - 4*s**2, 4*r - 4*r**2 - 4*r*s])
    dSdr = np.array([-3 + 4*r + 4*s, 4*r - 1, 0, 4*s, -4*s, 4 - 8*r - 4*s])
    dSds = np.array([-3 + 4*r + 4*s, 0, 4*s-1, 4*r, 4-4*r-8*s, -4*r])
    
    return S, dSdr, dSds


<h3> Mapowanie izoparametryczne </h3> 

In [5]:
def isoparametric_mapping(r, s, nodes):
    """
        Funkcja obliczająca komponenty mapowania izoparametrycznego dla punktu (r,s)
        z trójkąta referencyjnego mapowanego na dowolny trójkąt.
        input r,s - float(), float(), współrzędne punktu;
        nodes - np.array(), węzły trójkąta.
        output: S - np.array(),  wektor wartości funkcji kształtu w (r,s);
                dSdx - np.array(), wektor pochodnej czastkowej po 1. współrzędnej;
                dSdy - np.array(), wektor pochodnej czastkowej po 2. współrzędnej;
                detJ - float(),  Jakobian.
    """

    j = np.zeros([2, 2]) # rezerwujemy miejsce na macierz Jacobiego

    # Sprawdzamy czy mamy do czynienia z P1 czy P2 i liczymy outputy
    
    length = nodes.shape[0] 
    if length == 3:
        S, dSdr, dSds = P1shapes(r, s)
    elif length == 6:
        S, dSdr, dSds = P2shapes(r, s)
    else:
        print("Nieobsługiwany przypadek")
        return -1

    # Liczymy Jakobian
    
    j[0, 0] = np.dot(nodes[:, 0], dSdr)
    j[1, 0] = np.dot(nodes[:, 1], dSdr)
    j[0, 1] = np.dot(nodes[:, 0], dSds)
    j[1, 1] = np.dot(nodes[:, 1], dSds)
    
    detJ = j[0, 0] * j[1, 1] - j[1, 0] * j[0, 1]

    # Liczymy pochodne po x i y
    
    dSdx = (j[1, 1] * dSdr - j[1, 0] * dSds) / detJ
    dSdy = (- j[0, 1] * dSdr + j[0, 0] * dSds) / detJ

    return S, dSdx, dSdy, detJ


<h3> Gausspoints </h3>

In [6]:
def Gausspoints(precision):
    """
        Funkcja podająca wagi i punkty dla kwadratury Gaussa 2D dla trójkąta.
        input: precision - int,  poziom precyzji 1,2,3 lub 4;
        output: qwgts - np.array(),  wagi kwadratury;
                rspts - np.array(), punkty w których liczymy kwadraturę.
    """

    if precision == 1:
        qwgts = np.array([1])
        rspts = np.array([[1 / 3], [1 / 3]])
    elif precision == 2:
        qwgts = np.array([1 / 3, 1 / 3, 1 / 3])
        rspts = np.array([[1 / 6, 2 / 3, 1 / 6], [1 / 6, 1 / 6, 2 / 3]])
    elif precision == 3:
        qwgts = np.array([-27 / 48, 25 / 48, 25 / 48, 25 / 48])
        rspts = np.array([[1 / 3, 0.2, 0.6, 0.2], [1 / 3, 0.2, 0.2, 0.6]])
    elif precision == 4:
        qwgts = np.array([0.223381589678011,
                          0.223381589678011,
                          0.223381589678011,
                          0.109951743655322,
                          0.109951743655322,
                          0.109951743655322])
        rspts = np.array([[0.445948490915965, 0.445948490915965, 0.108103018168070, 0.091576213509771,
                           0.091576213509771, 0.816847572980459],
                          [0.445948490915965, 0.108103018168070, 0.445948490915965, 0.091576213509771,
                           0.816847572980459, 0.091576213509771]])
    else:
        print("Wybrano niepoprawną precyzję")

    return qwgts, rspts

<h3>Numeracja siatki dla P2</h3>

In [7]:
def Tri2Edge(P, T):
    """
        Funkcja licząca dodatkowe wierzchołki ( na środkach krawędzi ) dla trójkąta.
        input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
        output: edges - np.array(), macierz węzłów krawędziowych; int(len(v)/2) - int(), liczba dodanych węzłów
    """

    n = P.shape[0]  # liczba węzłów
    nt = T.shape[1]  # liczba trójkątów
    i = T[0, :]  # pierwszy wierzchołek dla każdego trójkąta
    j = T[1, :]  # drugi wierzchołek dla każdego trójkąta
    k = T[2, :]  # trzeci wierzchołek dla każdego trójkąta

    # Stworzymy macierz, która zaznaczy, pomiędzy którymi węzłami
    # ( reprezentowanymi przez współrzędne macierzy) jest węzeł
    idx_first = np.array([*j,*i,*i])
    idx_second = np.array([*k,*k,*j])

    A = scipy.sparse.lil_matrix((n, n), dtype=float)  # macierz występowania krawędzi

    for i in range(len(idx_first)):
        A[idx_first[i], idx_second[i]] = -1  # tam gdzie węzeł na krawędzi, to wstawimy -1
        A[idx_second[i], idx_first[i]] = -1  # węzeł pomiędzy a i b to też pomiędzy b i a

    v_ = A.nnz  # zgarniamy ilość niezerowych elementów macierzy A ( podwójna ilość węzłów krawędziowych )
    v = np.array(range(1, v_ + 1))  # ponumerujemy węzły krawędziowe, numerując od 1
    
    ind = A.nonzero()   # zgarniamy indeksy niezerowych atomów macierzy A
    
    # Numerujemy w pętli    
    k = 0
    for i in range(len(ind[0])):
        if ind[1][i] >= ind[0][i]:
            A[ind[0][i], ind[1][i]] = v[k]
            A[ind[1][i], ind[0][i]] = v[k]
            k = k + 1

    edges = np.zeros([nt, 3])  # inicjujemy macierz węzłów krawędziowych

    # Uzupełniamy macierz węzłów krawędziowych

    for k in range(nt):
        edges[k, 0] = A[
            T[1, k], T[2, k]]  # wierzchołek krawędziowy między drugim a trzecim wierzchołkiem
        edges[k, 1] = A[
            T[0, k], T[2, k]]  # wierzchołek krawędziowy między pierwszym a trzecim wierzchołkiem
        edges[k, 2] = A[
            T[0, k], T[1, k]]  # wierzchołek krawędziowy między pierwszym a drugim wierzchołkiem

    return edges, int(len(v) / 2)

def ChangeP1toP2Mesh(P, T):
    """
        Funkcja licząca macierz węzłów i macierz połączeń dla P2.
        input: P - np.array(), lista węzłów; T - np.array(), connection matrix.
        output: nodes - np.array(), macierz węzłów P2; connections - np.array(), macierz połączeń P2
    """

    n = P.shape[0]  # liczba węzłów
    edges, how_many_new_edges = Tri2Edge(P, T)  # uzyskujemy numery wierzchołków krawędziowych
    edges = edges + n - 1  # zmiany numeracji powyższych wierzchołków, jako kontynuacja tych dla P1
    # ( n-1 bo w Tri2Edge numerujemy od 1)
    i = T[0, :]
    j = T[1, :]
    k = T[2, :]

    extend_nodes = np.zeros([how_many_new_edges, 2])  # tworzymy miejsce
    # na uzupełnienie współrzędnych węzłów krawędziowych

    nodes = np.append(P, extend_nodes, axis=0)  # poszerzamy tablice wezlow o wezly wierzcholkowe
    connections = np.append(T, edges.T, axis=0).astype('int')  # poszerzamy tablice polaczen o wezly wierzcholkowe

    e = edges[:, 0].astype('int')  # bierzemy pierwsze wierzchołki krawędziowe dla każdego z trójkątów
    nodes[e, 0] = 0.5 * (nodes[j, 0] + nodes[k, 0])  # obliczamy ich współrzędne
    nodes[e, 1] = 0.5 * (nodes[j, 1] + nodes[k, 1])
    e = edges[:, 1].astype('int')  # bierzemy drugie węzły krawędziowe, itd.
    nodes[e, 0] = 0.5 * (nodes[i, 0] + nodes[k, 0])
    nodes[e, 1] = 0.5 * (nodes[i, 1] + nodes[k, 1])
    e = edges[:, 2].astype('int')
    nodes[e, 0] = 0.5 * (nodes[i, 0] + nodes[j, 0])
    nodes[e, 1] = 0.5 * (nodes[i, 1] + nodes[j, 1])

    return nodes, connections
